# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355462434714                   -1.10    6.1    193ms
  2   -9.356744017339       -2.89       -1.42    1.0    100ms
  3   -9.356975006727       -3.64       -2.79    3.8    122ms
  4   -9.357012166253       -4.43       -3.09    4.4    505ms
  5   -9.357012351344       -6.73       -3.24    5.4    146ms
  6   -9.357012445976       -7.02       -3.39    1.0   81.9ms
  7   -9.357012501258       -7.26       -3.54    2.2   98.5ms
  8   -9.357012525218       -7.62       -3.67    1.0   84.6ms
  9   -9.357012543678       -7.73       -3.97    1.0   87.2ms
 10   -9.357012547381       -8.43       -4.32    1.0   81.2ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      9
│     10
│     10
│     11
│     13
│      2
│      3
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74932471033                   -0.97    7.5    622ms
  2   -18.79114672106       -1.38       -1.31    2.4    266ms
  3   -18.79229364030       -2.94       -2.50    3.5    318ms
  4   -18.79240650219       -3.95       -2.91    5.1    445ms
  5   -18.79240821506       -5.77       -3.14    4.1    365ms
  6   -18.79240849021       -6.56       -3.46    1.8    238ms
  7   -18.79240858697       -7.01       -4.09    2.4    273ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55491815975                   -0.84    9.6    2.16s
  2   -37.56036756179       -2.26       -1.19    1.0    743ms
  3   -37.56180595861       -2.84       -2.65    5.5    1.14s
  4   -37.56453690027       -2.56       -2.46    8.6    1.86s
  5   -37.56453019063   +   -5.17       -2.38    8.1    1.18s
  6   -37.56455348201       -4.63       -3.60    1.0    735ms
  7   -37.56455379731       -6.50       -3.82   10.2    1.56s
  8   -37.56455407610       -6.55       -4.32    1.9    811ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355382636798                   -1.09    7.0    201ms
  2   -9.356782568714       -2.85       -1.91    1.0   63.6ms
  3   -9.357004189663       -3.65       -2.90    4.6    145ms
  4   -9.357002592886   +   -5.80       -2.76    3.6    156ms
  5   -9.357012011208       -5.03       -3.38    1.2   71.5ms
  6   -9.357012513143       -6.30       -4.09    1.1    351ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55521050315                   -0.84    8.2    1.92s
  2   -37.55828085134       -2.51       -1.62    1.0    967ms
  3   -35.08798791977   +    0.39       -0.87    8.0    1.53s
  4   -37.50663845289        0.38       -1.65    5.8    1.26s
  5   -37.54931692172       -1.37       -1.87    1.8    746ms
  6   -37.53926655461   +   -2.00       -1.85    3.8    1.21s
  7   -37.46446209766   +   -1.13       -1.56    4.9    1.10s
  8   -37.56428297615       -1.00       -2.51    4.0    963ms
  9   -37.56446832075       -3.73       -2.77    5.5    1.21s
 10   -37.56443846069   +   -4.52       -2.70    2.5    973ms
 11   -37.56454144859       -3.99       -3.13    1.6    1.05s
 12   -37.56454723571       -5.24       -3.37    3.2    1.02s
 13   -37.56455153523       -5.37       -3.53    1.8    817ms
 14   -37.56455278961       -5.90       -3.70    1.1    716ms
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.